In [13]:
import evaluation
import os
import tensorflow as tf
import pandas as pd
import numpy as np

import imp
imp.reload(evaluation)

<module 'evaluation' from '/Users/admin/Documents/NAD-model/evaluation.py'>

In [14]:
TEST_PATH = './datasets/pd-test'
TRAIN_PATH = './datasets/pd-train'

train_datas, train_labels = evaluation.read_csv(TRAIN_PATH)
test_datas, test_labels = evaluation.read_csv(TEST_PATH)
model_names = sorted(os.listdir("./params"))

In [15]:
model_names

['1ep-1024bs-5cv-model_vanilla_do_0.5-fe.txt']

In [16]:
for m in model_names:
  path = "./params/" + m

  with open(path, "r") as file:
    model_name = file.readline().split(": ")[1].strip("\n") + ".h5"
    ms = file.readline().split(": ")[1].strip("\n")
    eps = file.readline().split(": ")[1].strip("\n")
    thresh = file.readline().split(": ")[1].strip("\n")

  with tf.device('/GPU:0'):
    # batch detection
    train_res_batch = evaluation.batch_evaluate(train_datas, train_labels, "./models/" + model_name, [ float(eps), int(ms), float(thresh) ])
    test_res_batch = evaluation.batch_evaluate(test_datas, test_labels, "./models/" + model_name, [ float(eps), int(ms), float(thresh) ])
    
    # real-time detection
    train_res_rt = evaluation.real_time_evaluate(train_datas, train_labels, "./models/" + model_name, float(thresh))
    test_res_rt = evaluation.real_time_evaluate(test_datas, test_labels, "./models/" + model_name, float(thresh))
  
  df = pd.DataFrame(np.concatenate((train_res_batch, test_res_batch, train_res_rt, test_res_rt)))
  df = df.transpose()
  df.to_csv("./results/" + model_name.strip("h5") + ".csv", index=False, header=False)
  
  print(model_name + " success")

: 

: 